In [1]:
%load_ext autoreload
%autoreload 2
#---------- Library Imports ----------
import torch
import cv2
import numpy as np
from PIL import Image
from torchvision import transforms
import os
import sequential_creator
import glob
from torchmetrics import MeanAbsolutePercentageError as MAPE
import matplotlib as mpl
import re
import json
from tqdm import tqdm
import pytorch_lightning as pl
from torch.utils.data import DataLoader

c:\Users\Ulric\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%reload_ext autoreload

In [3]:
#---------- Personal python files imports ----------
import load_lettuce_dataset
import data_augmentation
import model_and_training_files
import lettuce_dataset

In [4]:
#---------- Hyperparameters ----------
batch_size = 16
learning_rate = 1e-3

In [5]:
#---------- Other parameters ----------
augmented_dataset_size = 20 # Size of the augmented dataset, so if the original dataset contained 103 images, they would be augmented and made into 2000 images
                              # One thing to note is that if this number is much bigger than the size of the original dataset, then they would most likely end up being duplicates, since there is not that many augmentations implemented at the moment

In [6]:
#---------- Load Lettuce Dataset ----------
rgb_list, depth_list, fresh_weight_list, dry_weight_list = load_lettuce_dataset.load_all_images()

100%|██████████| 341/341 [01:48<00:00,  3.13it/s]


In [9]:
#---------- Augment Lettuce Dataset ----------
depth_img_aug, rgb_imgs_aug, fresh_weight_GT, dry_weight_GT = data_augmentation.augment_data(rgb_images=rgb_list, depth_images=depth_list, fresh_weight_GT=fresh_weight_list, dry_weight_GT=dry_weight_list, amount_of_augmentated_images=augmented_dataset_size)

Augmenting RGB Images


100%|██████████| 20/20 [00:00<00:00, 42.37it/s]

Amount of augmented images 20


In [15]:
#---------- Create data loaders ----------
# Concatenate the depth and rgb images
full_dataset = []
full_dataset_labels = []
for i in range(augmented_dataset_size):
    depth_plus_rgb = np.concatenate((depth_img_aug[i], rgb_imgs_aug[i]), axis=0)
    full_dataset.append(depth_plus_rgb)
    #full_dataset.append([depth_img_aug[i], rgb_imgs_aug[i]])
    full_dataset_labels.append([fresh_weight_GT[i], dry_weight_GT[i]])

print("Size of images in the dataset: ", np.array(full_dataset).shape, " --> (Amount of images, total channels, img_w, img_h)")
print("Size of images in the dataset: ", np.array(full_dataset[0][0]).shape)

# Define the dataset
dataset = lettuce_dataset.LettuceDataset(full_dataset, full_dataset_labels)

# Split the dataset in train, validation and test
splitted_data = lettuce_dataset.data_splittage(augmented_dataset_size, [75, 12.5, 12.5])
print("train, validation, test = ", splitted_data)

train_set, validation_set, test_set = torch.utils.data.random_split(dataset, splitted_data)
#train_set, validation_set, test_set = torch.utils.data.random_split(dataset, [7500, 1250, 1250])

# Create dataloaders for train, validation and test data
train_loader = DataLoader(dataset = train_set, batch_size = batch_size, shuffle = True)
validation_loader = DataLoader(dataset = validation_set, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(dataset = test_set, batch_size = batch_size, shuffle = True)

Size of images in the dataset:  (20, 6, 224, 224)  --> (Amount of images, total channels, img_w, img_h)
Size of images in the dataset:  (224, 224)
train, validation, test =  [15, 2, 3]


In [9]:
#---------- Model Definition ----------
# Code modified from Anders Glent Buch elective course Deep Neural Networks
# Tip: dont forget to include activation layer (ReLu)

def build_cnn_layer(in_channels, out_channels, kernal_size=3, stride=1, padding=1, 
                    use_batchnorm=True, pool=False):
    '''
    in_channels: input channels to 2d convolution layer
    out_channels: ouput channels to 2d convolution layer
    kernal_size: kernal size (refer torch conv2d)
    stride: stride (refer torch conv2d)
    padding: padding (refer torch conv2d)  
    use_batchnorm: enable/disable batchnorm (refer torch BatchNorm2d)
    pool: enable/disable pooling (refer torch MaxPool2D)

    should return the built CNN layer
    '''
    return torch.nn.Sequential(
        torch.nn.Conv2d(in_channels, out_channels, kernal_size, stride=stride, padding=padding),
        torch.nn.ReLU(inplace=True),
        torch.nn.BatchNorm2d(out_channels) if use_batchnorm else torch.nn.Identity(),
        torch.nn.MaxPool2d(2, 2) if pool else torch.nn.Identity(),
    )

# Remember to flatten after convolution layers before its passed to linear layers
# Also activation layer for final layers
def build_cnn_model(pool=False):
    '''
    pool: enable/disable pooling

    should return built CNN model
    '''
    nf = 32
    s = 1 if pool else 2
    return torch.nn.Sequential(
        # (B, 3, 32, 32) images
        build_cnn_layer(3, nf),  # (B, nf, 32, 32)
        build_cnn_layer(nf, 2 * nf, stride=s, pool=pool),  # (B, 2 * nf, 16, 16)
        build_cnn_layer(2 * nf, 4 * nf, stride=s, pool=pool),  # (B, 4 * nf, 8, 8)
        build_cnn_layer(4 * nf, 8 * nf, stride=s, pool=pool),  # (B, 8 * nf, 4, 4)
        torch.nn.Flatten(), # (B, 8 * nf * 4 * 4)
        torch.nn.Linear(8 * nf * 4 * 4, 512), torch.nn.ReLU(True), torch.nn.BatchNorm1d(512), # (B, 512)
        torch.nn.Linear(512, 10), # (B, 10) logits
    )

Using cache found in C:\Users\Lasse/.cache\torch\hub\pytorch_vision_v0.10.0


Sequential(
  (0): Linear(in_features=2000, out_features=1000, bias=True)
  (1): ReLU()
  (2): Linear(in_features=1000, out_features=500, bias=True)
  (3): ReLU()
  (4): Linear(in_features=500, out_features=250, bias=True)
  (5): ReLU()
  (6): Linear(in_features=250, out_features=2, bias=True)
  (7): ReLU()
)


In [ ]:
#---------- Model declaration ----------
model = build_cnn_model()

In [12]:
#---------- Training ----------
print("Cuda available = ", torch.cuda.is_available())

# Define the model with Pytorch Lightning
model = model_and_training_files.BiomassModel(model=model,
                                                lr=learning_rate,
                                                )

trainer = model_and_training_files.get_trainer()    # gets the trainer (which is a class that takes the model and dataset)
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=validation_loader) # Train the model
trainer.test(model, dataloaders=test_loader)

#---------- Save the weights ----------
torch.save(model, "saved_models/best_weights_V1.plk") # Saves the regression head

Auto select gpus: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Cuda available =  True
1
Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
2
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
3
ReLU(inplace=True)
4
MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
5
Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (downsample): Sequential(
      (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type       | Params
--------------------------------------
0 | layers | Sequential | 2.6 M 
1 | Resnet | ResNet     | 44.5 M
--------------------------------------
45.7 M    Trainable params
1.4 M     Non-trainable params
47.2 M    Total params
188.706   Total estimated model params size (MB)


Epoch 199: 100%|██████████| 4/4 [00:03<00:00,  1.31it/s, loss=0.539, v_num=11]

`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 199: 100%|██████████| 4/4 [00:12<00:00,  3.14s/it, loss=0.539, v_num=11]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  2.45it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           1.1976007223129272
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [11]:
model = torch.load("saved_models/best_weights_V1.plk")
#fresh_weight_GT, dry_weight_GT
imgidx = 6
pred = model.prediction(depth_img_aug[imgidx], rgb_imgs_aug[imgidx])
print(pred)
print(fresh_weight_GT[imgidx], dry_weight_GT[imgidx])
loss = MAPE()
true = torch.from_numpy(np.array([[fresh_weight_GT[imgidx], dry_weight_GT[imgidx]]]))
print(loss(pred,true))

tensor([[0.0724, 0.0000]])
27.9 1.76
tensor(0.9987)
